In [18]:
import pandas as pd
import numpy as np
from utils import *
from data_processing import *

# load general data
general_pacific = pd.read_csv('champions-tour-2024-pacific-kickoff_data\general_data_champions-tour-2024-pacific-kickoff.csv')
general_americas = pd.read_csv('champions-tour-2024-americas-kickoff_data\general_data_champions-tour-2024-americas-kickoff.csv')
general_emea = pd.read_csv('champions-tour-2024-emea-kickoff_data\general_data_champions-tour-2024-emea-kickoff.csv')

# performance data
performance_pacific = pd.read_csv('champions-tour-2024-pacific-kickoff_data\performance_data_champions-tour-2024-pacific-kickoff.csv')
performance_americas = pd.read_csv('champions-tour-2024-americas-kickoff_data\performance_data_champions-tour-2024-americas-kickoff.csv')
performance_emea = pd.read_csv('champions-tour-2024-emea-kickoff_data\performance_data_champions-tour-2024-emea-kickoff.csv')

# economic data
economy_pacific = pd.read_csv('champions-tour-2024-pacific-kickoff_data\economy_data_champions-tour-2024-pacific-kickoff.csv')
economy_americas = pd.read_csv('champions-tour-2024-americas-kickoff_data\economy_data_champions-tour-2024-americas-kickoff.csv')
economy_emea = pd.read_csv('champions-tour-2024-emea-kickoff_data\economy_data_champions-tour-2024-emea-kickoff.csv')

# picks and bans
pick_ban_pacific = pd.read_csv('champions-tour-2024-pacific-kickoff_data\pick_ban_data_champions-tour-2024-pacific-kickoff.csv')
pick_ban_americas = pd.read_csv('champions-tour-2024-americas-kickoff_data\pick_ban_data_champions-tour-2024-americas-kickoff.csv')
pick_ban_emea = pd.read_csv('champions-tour-2024-emea-kickoff_data\pick_ban_data_champions-tour-2024-emea-kickoff.csv')

<h1>Individual Data</h1>
<h2>General Data</h2>

In [10]:
df_pacific_general = general_feature_creation_for_teams(general_pacific)
df_americas_general = general_feature_creation_for_teams(general_americas)
df_emea_general = general_feature_creation_for_teams(general_emea)

In [11]:
df_general = create_dataframe(df_emea_general,df_americas_general,df_pacific_general)
# Assuming the last column of the data is the target variable (e.g., region labels)
X_general, y_general = df_general.iloc[:, :-1], df_general.index.get_level_values(0)   # Features & Target variable (region labels)

<h3>Feature Selection & Visualization General Data</h3>

In [4]:
k_best_feature_general = selectKbest(X_general, y_general)

Selected Features:
Index(['avrg_acs_per_team_atk', 'std_d_per_team', 'std_d_per_team_atk',
       'std_d_per_team_dfs', 'std_adr_per_team', 'std_hs%_per_team',
       'std_fk_per_team', 'avrg_fk_per_team_atk', 'std_fk_per_team_atk',
       'avrg_fk_per_team_dfs'],
      dtype='object')


In [16]:
k_best_feature_rfecv = RFECV_feature_selection(X_general,y_general,"linear")

Selected Features:
Index(['std_r_per_team_dfs', 'std_k_per_team', 'std_d_per_team',
       'std_d_per_team_atk', 'std_adr_per_team', 'avrg_hs%_per_team_atk',
       'std_hs%_per_team_atk', 'std_fk_per_team', 'avrg_fk_per_team_atk',
       'std_fk_per_team_atk', 'avrg_fk_per_team_dfs'],
      dtype='object')
Optimal number of features: 11


In [5]:
visualize_mean_feature_for_each_region(df_general,k_best_feature_general)

Plots saved as PNG files.


<h2>Performance data</h2>

In [6]:
performance_feature_emea = performance_feature_creation_for_teams(performance_emea, economy_emea)
performance_feature_americas = performance_feature_creation_for_teams(performance_americas, economy_americas)
performance_feature_pacific = performance_feature_creation_for_teams(performance_pacific, economy_pacific)

In [8]:
df_performance = create_dataframe(performance_feature_emea,performance_feature_americas,performance_feature_pacific)
columns_to_drop = ['5k_mean', '5k_std','1v5_mean', '1v5_std']
df_performance_dropped = df_performance.drop(columns=columns_to_drop)
X_performance, y_performance = df_performance_dropped.iloc[:, :-1], df_performance_dropped.index.get_level_values(0)   # Features & Target variable (region labels)

In [4]:
k_best_feature_performance = selectKbest(X_performance, y_performance, k=10)

Feature '2k_mean': 0.561182270618906
Feature '2k_std': 0.7833969591481695
Feature '3k_std': 0.47117442518602237
Feature '4k_mean': 1.1873820438640228
Feature '1v1_std': 0.6444426610886786
Feature '1v2_mean': 0.6237527212317263
Feature '1v2_std': 0.5614204568696978
Feature '1v3_std': 0.6534339325305514
Feature 'econ_std': 0.7659081599989729
Feature 'pl_mean': 0.9437653919871462


In [17]:
k_best_feature_rfecv_performance = RFECV_feature_selection(X_performance,y_performance,"linear")

Selected Features:
Index(['2k_mean', '2k_std', '3k_std', '4k_mean', '1v1_mean', '1v1_std',
       '1v2_mean', '1v2_std', '1v3_mean', '1v3_std', 'pl_mean', 'pl_std',
       'de_mean'],
      dtype='object')
Optimal number of features: 13


In [9]:
visualize_mean_feature_for_each_region(df_performance,k_best_feature_performance)

Plots saved as PNG files.


<h2>Economic Data</h2>

In [58]:
al = calculate_mean_economy(economy_pacific)

In [33]:
# Bank & Buys

# 
t = create_summary_rounds_dataset(economy_pacific)

In [54]:
test = t.columns[1:][:-1]

In [55]:
test

Index(['Pistol_Won', 'Pistol', 'Eco_Won', 'Eco', '$_Won', '$', '$$_Won', '$$',
       '$$$_Won', '$$$'],
      dtype='object')

In [27]:
ratio = create_ratio_economy_rounds(t)

In [31]:
ratio.set_index('Team Name')
columns_to_drop = ['Pistol_Won', 'Pistol', 'Eco_Won', 'Eco', '$_Won', '$',
       '$$_Won', '$$', '$$$_Won', '$$$']
dropped_ratio = ratio.drop(columns=columns_to_drop)

In [32]:
dropped_ratio

,Team Name,nb_rounds,ratio_pistol_won,ratio_eco_won,ratio_$_won,ratio_$$_won,ratio_$$$_won,ratio_Eco,ratio_$,ratio_$$,ratio_$$$
0,DRX,133,0.75,0.56,0.25,0.53,0.56,0.135,0.030,0.241,0.594
1,GEN,429,0.52,0.39,0.38,0.60,0.57,0.142,0.061,0.245,0.552
2,TLN,122,0.58,0.40,0.40,0.45,0.53,0.164,0.041,0.238,0.557
3,PRX,191,0.39,0.28,0.00,0.53,0.62,0.152,0.052,0.246,0.550
4,GE,149,0.36,0.25,0.00,0.46,0.49,0.161,0.034,0.262,0.544
5,RRQ,136,0.42,0.32,0.17,0.47,0.53,0.140,0.044,0.316,0.500
6,DFM,86,0.25,0.15,0.00,0.44,0.49,0.151,0.058,0.314,0.477
7,ZETA,212,0.59,0.45,0.08,0.43,0.50,0.146,0.057,0.264,0.533
8,T1,163,0.50,0.36,0.00,0.54,0.61,0.135,0.037,0.215,0.613
9,BLD,122,0.50,0.33,0.17,0.39,0.52,0.123,0.049,0.230,0.598


In [63]:
def economy_feature_creation_for_teams(economy):
    """ Function that create the dataframe with selected feature for the economic data for each region """
    
    # Economic rounds ratio
    summary_economy_rounds = create_summary_rounds_dataset(economy)
    columns_to_drop = summary_economy_rounds.columns[1:][:-1]
    ratio = create_ratio_economy_rounds(summary_economy_rounds)
    ratio_dropped = ratio.drop(columns=columns_to_drop)
    ratio_dropped.set_index(keys=['Team Name'])

    # Bank and Buys
    #bank_and_buys = calculate_mean_economy(economy)
    #ratio_dropped['Bank'] = ratio_dropped.index.map(lambda x: bank_and_buys[x][0])
    #ratio_dropped['Buys'] = ratio_dropped.index.map(lambda x: bank_and_buys[x][1])

    return ratio_dropped
    

r = economy_feature_creation_for_teams(economy_pacific)

In [64]:
r

,Team Name,nb_rounds,ratio_pistol_won,ratio_eco_won,ratio_$_won,ratio_$$_won,ratio_$$$_won,ratio_Eco,ratio_$,ratio_$$,ratio_$$$
0,DRX,133,0.75,0.56,0.25,0.53,0.56,0.135,0.030,0.241,0.594
1,GEN,429,0.52,0.39,0.38,0.60,0.57,0.142,0.061,0.245,0.552
2,TLN,122,0.58,0.40,0.40,0.45,0.53,0.164,0.041,0.238,0.557
3,PRX,191,0.39,0.28,0.00,0.53,0.62,0.152,0.052,0.246,0.550
4,GE,149,0.36,0.25,0.00,0.46,0.49,0.161,0.034,0.262,0.544
5,RRQ,136,0.42,0.32,0.17,0.47,0.53,0.140,0.044,0.316,0.500
6,DFM,86,0.25,0.15,0.00,0.44,0.49,0.151,0.058,0.314,0.477
7,ZETA,212,0.59,0.45,0.08,0.43,0.50,0.146,0.057,0.264,0.533
8,T1,163,0.50,0.36,0.00,0.54,0.61,0.135,0.037,0.215,0.613
9,BLD,122,0.50,0.33,0.17,0.39,0.52,0.123,0.049,0.230,0.598
